# Project Scenario:

I have been hired as a data engineer by research organization. My boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

My job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

In [4]:
# importing necessary libraries

import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import html5lib
from datetime import datetime 
import sqlite3

In [5]:
table_attribs = ["Name", "MC_USD_Billion"]#
db_name = 'Banks.db' #establishing the database name
table_name = 'Largest_banks'#establishing the table name
csv_path = './Largest_banks_data.csv'#establishing the csv file path
url="https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"#setting the URL a veriable name
efx_path="exchange_rate.csv"

In [6]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    data=requests.get(url).text
    soup=BeautifulSoup(data)
    df=pd.DataFrame(columns=table_attribs)
    table=soup.find_all("tbody")#gettting all the tables on the website and setting it to the tables variable
    rows=table[0].find_all("tr")#getting the table we need which is the second table on the website, you can manually inspect the website to get th
    for row in rows:#looping over rows
        col= row.find_all("td")# Find all the cells (columns) within the current row
        if len(col) != 0: #a condition to filter out cells that are empty
            if col[1].find_all("a")[1].contents[0]:
               data_dict ={"Name":col[1].find_all("a")[1].contents[0],"MC_USD_Billion":col[2].contents[0].strip()}#getting the first element within the  col[0] & col[2]
               df2=pd.DataFrame(data_dict,index=[0])
               df=pd.concat([df,df2],ignore_index= True)
    return df



def transform(df,efx_path):
    fx_df=pd.read_csv(efx_path)
    df["MC_USD_Billion"] = pd.to_numeric(df["MC_USD_Billion"], errors="coerce")
    df["MC_EUR_Billion"]=round(df["MC_USD_Billion"] *fx_df.at[0,"Rate"],2)
    df["MC_GBP_Billion"]=round(df["MC_USD_Billion"] *fx_df.at[1,"Rate"],2)
    df["MC_INR_Billion"]=round(df["MC_USD_Billion"] *fx_df.at[2,"Rate"],2)
    
    return df

def load_to_csv(df,csv_path):
    '''this function recieves two argument the data frame and the file path where it will save the csv'''
    df.to_csv(csv_path)
    
def load_to_db(df,conn,table_name):
    '''this function takes in 3 argument dataframe ,sql_connection to establish connection to the sqlite
    database and the table_name'''
    df.to_sql(table_name,conn,if_exists='replace',index=False)
    
def run_query(conn, query_statements):
    ''' This function runs the stated queries on the database table and
    prints the output of each query on the terminal. 
    The function accepts a database connection and a list of query statements. '''

    for query in query_statements:
        print(f"Executing query: {query}")
        query_output = pd.read_sql(query, conn)
        print(query_output)
        print("\n")
        
        


    
def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''

    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')   
        
        
        
conn =sqlite3.connect("Banks.db")

query_statements = [
    f"SELECT * FROM {table_name}",
    f"SELECT AVG(MC_GBP_Billion) FROM {table_name}",
    f"SELECT Name from {table_name} LIMIT 5"
    
   
]
    
    

    
    
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df,efx_path)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

log_progress('SQL Connection initiated.')

load_to_db(df, conn, table_name)

log_progress('Data loaded to Database as table. Running the query')


run_query(conn, query_statements)

log_progress('Process Complete.')

conn.close()

    

Executing query: SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3               Agricultural Bank of China          160.68          149.43   
4                                HDFC Bank          157.91          146.86   
5                              Wells Fargo          155.87          144.96   
6                        HSBC Holdings PLC          148.90          138.48   
7                           Morgan Stanley          140.83          130.97   
8                  China Construction Bank          139.82          130.03   
9                            Bank of China          136.81          127.23   

   MC_GBP_Billion  MC_INR_Billion  
0          346.34        35910.71  
1          185.22       

Experiencing a series of trials and errors paved the way for achieving the desired outcome at the summit. I felt compelled to share this here, underscoring the array of challenges faced and the lessons learned from overcoming errors encountered along the journey.

In [11]:
df

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39


In [12]:
cur= conn.cursor()

In [16]:
res=cur.execute("select * from Largest_banks")
res.fetchall()

[('JPMorgan Chase', 432.92, 402.62, 346.34, 35910.71),
 ('Bank of America', 231.52, 215.31, 185.22, 19204.58),
 ('Industrial and Commercial Bank of China', 194.56, 180.94, 155.65, 16138.75),
 ('Agricultural Bank of China', 160.68, 149.43, 128.54, 13328.41),
 ('HDFC Bank', 157.91, 146.86, 126.33, 13098.63),
 ('Wells Fargo', 155.87, 144.96, 124.7, 12929.42),
 ('HSBC Holdings PLC', 148.9, 138.48, 119.12, 12351.26),
 ('Morgan Stanley', 140.83, 130.97, 112.66, 11681.85),
 ('China Construction Bank', 139.82, 130.03, 111.86, 11598.07),
 ('Bank of China', 136.81, 127.23, 109.45, 11348.39)]

In [10]:
data=requests.get(url).text
soup=BeautifulSoup(data)
df=pd.DataFrame(columns=table_attribs)
table=soup.find_all("tbody")
rows=table[1].find_all("tr")
for row in rows:
    col= row.find_all("td")
    if len(col) != 0:
        data_dict ={"Name":col[1].a.contents[0],"MC_USD_Billion":col[2].contents[0]}
        df2 =pd.DataFrame(data_dict, index=[0])
        df=pd.concat([df,df2],ignore_index=True)

ValueError: Length of values (0) does not match length of index (1)

In [63]:
data=requests.get(url).text
soup=BeautifulSoup(data)
df=pd.DataFrame(columns=table_attribs)
table=soup.find_all("tbody")
rows=table[0].find_all("tr")
for row in rows:
    col= row.find_all("td")
    if len(col) != 0:
        if col[1].find_all("a")[1].contents[0]:
           data_dict ={"Name":col[1].find_all("a")[1].contents[0],"MC_USD_Billion":col[2].contents[0].strip()}
           df2=pd.DataFrame(data_dict,index=[0])
           df=pd.concat([df,df2],ignore_index= True)

In [60]:
data_dict

{'Name': 'Bank of China', 'MC_USD_Billion': '136.81'}

In [64]:
df

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [5]:
table

[<tbody><tr>
 <th data-sort-type="number">Rank
 </th>
 <th>Bank name
 </th>
 <th>Market cap<br/>(US$ billion)
 </th></tr>
 <tr>
 <td>1
 </td>
 <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908

In [8]:
col

[<td>10
 </td>,
 <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/China" title="China"><img alt="China" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/https://en.wikipedia.org/wiki/Bank_of_C

In [55]:
data=requests.get(url).text
soup=BeautifulSoup(data)
df=pd.DataFrame(columns=table_attribs)
table=soup.find_all("tbody")
rows=table[1].find_all("tr")
    col= row.find_all("td")
    for column in col:
        x=column.find_all("href")

In [40]:
col[1]

<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/China" title="China"><img alt="China" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/https://en.wikipedia.org/wiki/Bank_of_China" title="Bank

In [45]:
col[1].find_all("a")[1]

<a href="/web/20230908091635/https://en.wikipedia.org/wiki/Bank_of_China" title="Bank of China">Bank of China</a>

In [46]:
col[1].a[1].contents[0]

KeyError: 1

In [47]:
col[1].find_all("a")[1].contents[0]

'Bank of China'

In [58]:
col[2].contents[0].strip()

'136.81'

In [57]:
col[2].strip()

TypeError: 'NoneType' object is not callable

In [100]:
conn =sqlite3.connect("Banksss.db")
df.to_sql(table_name,conn,if_exists='replace',index=False)

10

In [101]:
 query_output = pd.read_sql(query_statement1,conn)

In [102]:
query_output

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [103]:
 query_output = pd.read_sql(query_statement2,conn)

In [104]:
query_output

,AVG(MC_USD_Billion)
0,189.982


In [109]:
query_loop =[f"SELECT * FROM {table_name}",f"SELECT AVG(MC_USD_Billion) FROM {table_name}",f"SELECT Name from {table_name} LIMIT 5"]


In [111]:
for query in query_loop:
    pd.read_sql(query_loop,conn)
    print(query)
    

TypeError: Query must be a string unless using sqlalchemy.

In [69]:
fx_df=pd.read_csv("exchange_rate.csv")
fx_df

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [70]:
df

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [71]:
dfc=df.copy()

In [87]:
dfc["MC_EUR_Billion"]=round(dfc["MC_USD_Billion"] *fx_df.at[1,"Rate"],2)


In [75]:
fx_df.loc[1,"Rate"]

0.8

In [74]:
fx_df

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [78]:
dfc["MC_USD_Billion"] = pd.to_numeric(dfc["MC_USD_Billion"], errors="coerce")


In [79]:
dfc

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [88]:
dfc

,Name,MC_USD_Billion,EUR
0,JPMorgan Chase,432.92,346.34
1,Bank of America,231.52,185.22
2,Industrial and Commercial Bank of China,194.56,155.65
3,Agricultural Bank of China,160.68,128.54
4,HDFC Bank,157.91,126.33
5,Wells Fargo,155.87,124.70
6,HSBC Holdings PLC,148.90,119.12
7,Morgan Stanley,140.83,112.66
8,China Construction Bank,139.82,111.86
9,Bank of China,136.81,109.45


In [ ]:
def run_queries(conn, query_statements):
  

    for query in query_statements:
        print(f"Executing query: {query}")
        query_output = pd.read_sql(query, conn)
        print(query_output)
        print("\n")
        
        
query_statements = [
    f"SELECT * FROM {table_name}",
    f"SELECT AVG(MC_USD_Billion) FROM {table_name}",
    f"SELECT Name from {table_name} LIMIT 5"
    
   
]
query_statement1=f"SELECT * FROM {table_name}"
query_statement2=f"SELECT AVG(MC_USD_Billion) FROM {table_name}"
query_statement3=f"SELECT Name from {table_name} LIMIT 5"
    